In [ ]:
#Inspect dataset

#import pandas as pd

#df_all = pd.read_csv("/Users/mariamorazamora/Desktop/Prototyping/Assignment 1/credit_card_transactions.csv")

#print("Shape:")
#print(df_all.shape)

#print("\nColumns:")
#print(df_all.columns)

#print("\nFirst rows:")
#print(df_all.head())

#print("\nInfo:")
#print(df_all.info())


Shape:
(1296675, 24)

Columns:
Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud', 'merch_zipcode'],
      dtype='object')

First rows:
   Unnamed: 0 trans_date_trans_time            cc_num  \
0           0   2019-01-01 00:00:18  2703186189652095   
1           1   2019-01-01 00:00:44      630423337322   
2           2   2019-01-01 00:00:51    38859492057661   
3           3   2019-01-01 00:01:16  3534093764340240   
4           4   2019-01-01 00:03:06   375534208663984   

                             merchant       category     amt      first  \
0          fraud_Rippin, Kub and Mann       misc_net    4.97   Jennifer   
1     fraud_Heller, Gutmann and Zieme    grocery_pos  107.23  Stephanie   
2                fraud_Lind-Buckridge  entertainment  220.11     Edward  

In [1]:
#Inspect filtered (only one person) dataset

import pandas as pd

df = pd.read_csv("transactions_user_24_ccnum_567868110212.csv")

print("Shape:")
print(df.shape)

print("\nColumns:")
print(df.columns)

print("\nFirst rows:")
print(df.head())

print("\nInfo:")
print(df.info())

Shape:
(2558, 9)

Columns:
Index(['trans_date_trans_time', 'cc_num', 'category', 'amt', 'merchant',
       'is_fraud', 'year', 'month', 'day'],
      dtype='object')

First rows:
  trans_date_trans_time        cc_num       category     amt  \
0   2019-01-01 17:03:45  567868110212   shopping_net    9.17   
1   2019-01-02 13:16:51  567868110212      kids_pets   30.87   
2   2019-01-02 23:03:27  567868110212   shopping_pos   60.39   
3   2019-01-03 15:00:35  567868110212  personal_care  134.95   
4   2019-01-03 15:54:54  567868110212   shopping_pos    2.50   

                merchant  is_fraud  year  month  day  
0  fraud_Kerluke-Abshire         0  2019      1    1  
1      fraud_Ullrich Ltd         0  2019      1    2  
2       fraud_Dooley Inc         0  2019      1    2  
3      fraud_Reichel LLC         0  2019      1    3  
4   fraud_Torphy-Goyette         0  2019      1    3  

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2558 entries, 0 to 2557
Data columns (total 9 col

In [ ]:
#Sanity check (should be only 1 user)

n_users = df["cc_num"].nunique()
print(n_users)


1


In [6]:
#Transform dataset acording to my needs

#convert date
df["trans_date_trans_time"] = pd.to_datetime(df["trans_date_trans_time"])

#extract info from date
df["year"] = df["trans_date_trans_time"].dt.year
df["month"] = df["trans_date_trans_time"].dt.month
df["day"] = df["trans_date_trans_time"].dt.day

#amount of users
#df["cc_num"].nunique() #983

#choose one random user and work with that
#user_id = df["cc_num"].iloc[24]
#df_user = df[df["cc_num"] == user_id].copy()
#since now the db is already filtered:
df_user = df.copy()

#amount of months of that user
df_user.groupby(["year", "month"]).size()

#categories
df_user["category"].value_counts()



category
shopping_pos      297
grocery_pos       251
shopping_net      250
home              228
kids_pets         217
personal_care     196
food_dining       189
misc_pos          176
health_fitness    172
entertainment     165
grocery_net       153
misc_net          130
gas_transport      85
travel             49
Name: count, dtype: int64

In [7]:
#Now that we now that the slected user has enough data, I will clean the dataset 

category_map = {
    "grocery_pos": "Groceries",
    "grocery_net": "Groceries",
    "food_dining": "Eating out",
    "entertainment": "Leisure",
    "travel": "Leisure",
    "shopping_pos": "Shopping",
    "shopping_net": "Shopping",
    "gas_transport": "Transport",
    "home": "Home",
    "personal_care": "Personal care",
    "health_fitness": "Health",
    "kids_pets": "Kids & Pets",
    "misc_pos": "Misc",
    "misc_net": "Misc"
}

df_user["budget_category"] = df_user["category"].map(category_map)

In [8]:
#decide on a random current date

CURRENT_YEAR = 2020
CURRENT_MONTH = 6

df_train = df_user[
    (df_user["year"] < CURRENT_YEAR) |
    ((df_user["year"] == CURRENT_YEAR) & (df_user["month"] < CURRENT_MONTH))
].copy()

df_current = df_user[
    (df_user["year"] == CURRENT_YEAR) &
    (df_user["month"] == CURRENT_MONTH)
].copy()

In [9]:
#create consumption curve

# Daily spend
daily_train = (
    df_train
    .groupby(["year","month","budget_category","day"])["amt"]
    .sum()
    .reset_index()
)

# Monthly totals
monthly_totals = (
    daily_train
    .groupby(["year","month","budget_category"])["amt"]
    .sum()
    .reset_index()
    .rename(columns={"amt":"month_total"})
)

daily_train = daily_train.merge(
    monthly_totals,
    on=["year","month","budget_category"]
)

daily_train["cum_spend"] = (
    daily_train
    .sort_values("day")
    .groupby(["year","month","budget_category"])["amt"]
    .cumsum()
)

daily_train["cum_frac"] = daily_train["cum_spend"] / daily_train["month_total"]

# Average curve
avg_curve = (
    daily_train
    .groupby(["budget_category","day"])["cum_frac"]
    .mean()
    .reset_index()
)


In [10]:
#Forecast for the current month

DAY_FOR_FORECAST = 15

current_spent = (
    df_current[df_current["day"] <= DAY_FOR_FORECAST]
    .groupby("budget_category")["amt"]
    .sum()
    .reset_index()
    .rename(columns={"amt":"spent_so_far"})
)

forecast_rows = []

for _, row in current_spent.iterrows():
    cat = row["budget_category"]
    spent = row["spent_so_far"]

    frac_row = avg_curve[
        (avg_curve["budget_category"] == cat) &
        (avg_curve["day"] == DAY_FOR_FORECAST)
    ]

    if frac_row.empty:
        continue

    frac = frac_row["cum_frac"].iloc[0]
    forecast = spent / frac

    forecast_rows.append({
        "category": cat,
        "spent_so_far": spent,
        "forecast_end_month": forecast
    })

forecast_df = pd.DataFrame(forecast_rows)
print(forecast_df)



        category  spent_so_far  forecast_end_month
0     Eating out        296.30          541.299510
1      Groceries       2472.73         4687.460994
2         Health        534.49         1296.891988
3           Home        280.58          677.314542
4    Kids & Pets        335.61          611.184718
5        Leisure        557.12         1208.530787
6           Misc        439.46         1261.547706
7  Personal care        495.67         1081.552367
8       Shopping        689.70         1366.648191
9      Transport        172.06          304.018092
